In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50V2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [2]:
# Set random seed for reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
# Configuration parameters
IMG_SIZE = 224  # EfficientNet and ResNet prefer 224x224
BATCH_SIZE = 32
EPOCHS = 50
NUM_CLASSES = 39  # 38 plant disease classes + 1 random object class
LEARNING_RATE = 1e-4
DROPOUT_RATE = 0.5

In [4]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
# Only rescaling for validation set
valid_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Load datasets with augmentation
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_generator = valid_datagen.flow_from_directory(
    'valid',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 76764 images belonging to 39 classes.
Found 21062 images belonging to 39 classes.


In [7]:
# Get class names and indices
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())
print(f"Classes: {class_names}")
print(f"Total number of classes: {len(class_names)}")

Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Random___image', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider

In [8]:

# Build model based on transfer learning with EfficientNetB0
def build_model(base_model_name="efficientnet"):
    if base_model_name == "efficientnet":
        # Load pre-trained EfficientNetB0 with weights
        base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
            pooling='avg'
        )
    else:
        # Alternative: ResNet50V2
        base_model = ResNet50V2(
            weights='imagenet',
            include_top=False,
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
            pooling='avg'
        )
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Create the model
    model = Sequential([
        base_model,
        BatchNormalization(),
        Dense(1024, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(512, activation='relu'),
        Dropout(DROPOUT_RATE/2),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model



In [9]:
# Create callbacks for training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
]

In [10]:
# Create model
model = build_model()
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 1280)                │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 39)                  │          20,007 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,911,242 (22.55 MB)

 Trainable params: 1,859,111 (7.09 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [ ]:
# Train model (Phase 1 - with frozen base model)
history = model.fit(
    train_generator,
    epochs=15,  # Initial training with frozen base layers
    validation_data=valid_generator,
    callbacks=callbacks
)


C:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0709 - loss: 3.7053   

2399/2399 ━━━━━━━━━━━━━━━━━━━━ 3676s 2s/step - accuracy: 0.0709 - loss: 3.7053 - val_accuracy: 0.1657 - val_loss: 3.6389 - learning_rate: 1.0000e-04
Epoch 2/15
   3/2399 ━━━━━━━━━━━━━━━━━━━━ 52:13 1s/step - accuracy: 0.1007 - loss: 3.6097

In [ ]:
# Fine-tuning: Unfreeze some layers of the base model for fine-tuning
if isinstance(model.layers[0], tf.keras.Model):  # If using a base model
    base_model = model.layers[0]
    # Unfreeze the last 30 layers
    for layer in base_model.layers[-30:]:
        layer.trainable = True


In [ ]:

# Recompile with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [ ]:
# Continue training with unfrozen layers (Phase 2)
fine_tune_history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=callbacks,
    initial_epoch=len(history.history['loss'])  # Continue from where we left off
)



In [ ]:
# Plot training & validation accuracy/loss
def plot_training_history(history, fine_tune_history=None):
    # Extract history from each phase
    history_dict = history.history
    if fine_tune_history:
        for key in fine_tune_history.history:
            history_dict[key] = history_dict.get(key, []) + fine_tune_history.history[key]
    
    # Create a figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training & validation accuracy
    ax1.plot(history_dict['accuracy'])
    ax1.plot(history_dict['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='lower right')
    
    # Plot training & validation loss
    ax2.plot(history_dict['loss'])
    ax2.plot(history_dict['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()



In [ ]:
# Plot training history
plot_training_history(history, fine_tune_history)

# Evaluate the model
print("Evaluating model...")
evaluation = model.evaluate(valid_generator)
print(f"Loss: {evaluation[0]:.4f}, Accuracy: {evaluation[1]:.4f}")

# Generate predictions for the validation set
valid_generator.reset()
y_pred = model.predict(valid_generator, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)



In [ ]:
# Get true classes (need to find a way to get them since valid_generator doesn't have this directly)
true_classes = valid_generator.classes

# Print classification report
report = classification_report(true_classes, y_pred_classes, target_names=class_names)
print("\nClassification Report:\n", report)



In [ ]:
# Plot confusion matrix
plt.figure(figsize=(20, 15))
cm = confusion_matrix(true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()



In [ ]:
# Save the model
model.save("plant_disease_model.h5")
print("Model saved as plant_disease_model.h5")



In [ ]:
# Function to test the model on a sample image
def test_on_sample(model, img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    class_name = class_names[predicted_class]
    
    plt.imshow(img)
    plt.title(f"Predicted: {class_name} ({confidence:.2f})")
    plt.axis('off')
    plt.show()
    
    return class_name, confidence

# Test on a few sample images if you have them
# Uncomment and specify paths to test images
# test_on_sample(model, "path/to/test/image1.jpg")
# test_on_sample(model, "path/to/test/image2.jpg")